In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)


In [0]:
%%sh


aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136 -U app_bdp_usage_qa -d dna -p 6432 << EOF 

select * from dna_genre_id_product_mapping where product_id=1427744264 limit 10;
select count (*) from dna_genre_id_product_mapping;

EOF

In [0]:



from conf.settings import PG_DNA_NAME, PG_DNA_ACCESS_ID, PG_DNA_HOSTS, PG_DNA_SECRET_KEY
from applications.db_check_v1.common.db_check_utils import query_df

dna_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DNA_NAME,
        user=PG_DNA_ACCESS_ID,
        host=PG_DNA_HOSTS[0][0],
        password=PG_DNA_SECRET_KEY,
        port=PG_DNA_HOSTS[0][1]
    )
)
mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").toPandas()
sql = 'SELECT * FROM dna_genre_id_product_mapping'
mapping_df_raw = query_df(dna_dsn, sql)

mapping_df_raw['created_time'] = mapping_df_raw['created_time'].dt.strftime('%Y-%m-%d')
mapping_df_raw['last_updated_time'] = mapping_df_raw['last_updated_time'].dt.strftime('%Y-%m-%d')
mapping_df_raw['genre_id'] = [str(map(int, l))  if l else 'None' for l in mapping_df_raw['genre_id']]
mapping_df_raw['modifier_id'] = [str(map(int, l)) if l else 'None' for l in mapping_df_raw['modifier_id']]

mapping_df_unified['genre_id'] = mapping_df_unified['genre_id'].astype("str")
mapping_df_unified['modifier_id'] = mapping_df_unified['modifier_id'].astype("str")
print mapping_df_raw
print mapping_df_unified


In [0]:


def _compare_df(df1, df2, on=None):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
        diff_df = diff_df.loc[diff_df["_merge"] != "both"]
        if len(diff_df) != 0:
            print diff_type
            print diff_df


# print mapping_df_raw.genre_id.dtypes
# print '*'*100
# print mapping_df_unified.genre_id.dtypes
# print mapping_df_raw
# print mapping_df_unified

_compare_df(mapping_df_raw, mapping_df_unified, on=["product_id", "created_time", "created_by", "last_updated_time", "last_updated_by", "comments", "modifier_id", "genre_id"])
print "pass"


In [0]:


est_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date=2019-12-27/")
est_unified_df.show()
# print est_unified_df
# print set(est_unified_df.device_code.tolist())
# print set(est_unified_df.country_code.tolist())
# print set(est_unified_df.genre_id.tolist())
# print set(est_unified_df.modifier_id.tolist())



In [0]:

store_est_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2019-12-27/")
mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/")

transformed_mapping_df = mapping_df_unified.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id'))
transformed_store_est_unified_df = store_est_unified_df.select(["app_id", "device_code", "country_code", "free_app_download", "paid_app_download", "revenue"]).withColumnRenamed("app_id", "product_id")




In [0]:

transformed_store_est_unified_df.show(5)
transformed_mapping_df.show(5)
transformed_store_est_unified_df.withColumn('total_col', transformed_store_est_unified_df.free_app_download + transformed_store_est_unified_df.paid_app_download).filter("free_app_download=0 or paid_app_download=0").show(5)



In [0]:


from pyspark.sql import functions as F

giq_df = transformed_store_est_unified_df.join(transformed_mapping_df, transformed_mapping_df.product_id == transformed_store_est_unified_df.product_id, how='inner').groupBy(["device_code", "country_code", "genre_id"]).agg({
            "free_app_download": "sum",
            "paid_app_download": "sum",
            "revenue": "sum",
        }).withColumnRenamed("sum(free_app_download)", "free_app_download").withColumnRenamed("sum(paid_app_download)", "paid_app_download").withColumnRenamed("sum(revenue)", "revenue")


giq_df = giq_df.fillna({"free_app_download":0, "paid_app_download":0})
# giq_df = giq_df.withColumn("download", F.when(giq_df.free_app_download + giq_df.paid_app_download>0, giq_df.free_app_download + giq_df.paid_app_download).otherwise(F.lit(None)))
giq_df = giq_df.withColumn("download", giq_df.free_app_download + giq_df.paid_app_download)






# def replace(column, value):
#     return when(column != value, column).otherwise(lit(None))

# giq_df.withColumn("download", replace(col("y"), "bar")).show()




In [0]:

s1=giq_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])
s2=est_unified_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])

diff = s1.union(s2).subtract(s1.intersect(s2))
diff.show()



print diff.count()
print s1.count()
print s2.count()

# |  ios-phone|          NA|     207|       8|   null|



In [0]:


import pandas as pd

mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/")
transformed_mapping_df = mapping_df_unified.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id'))

def compare(date):
    #collect
    store_est_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))
    est_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date={}/".format(date))
    
    #transform
    transformed_store_est_unified_df = store_est_unified_df.select(["app_id", "device_code", "country_code", "free_app_download", "paid_app_download", "revenue"]).withColumnRenamed("app_id", "product_id")
    giq_df = transformed_store_est_unified_df.join(transformed_mapping_df, transformed_mapping_df.product_id == transformed_store_est_unified_df.product_id, how='inner').groupBy(["device_code", "country_code", "genre_id"]).agg({
            "free_app_download": "sum",
            "paid_app_download": "sum",
            "revenue": "sum",
        }).withColumnRenamed("sum(free_app_download)", "free_app_download").withColumnRenamed("sum(paid_app_download)", "paid_app_download").withColumnRenamed("sum(revenue)", "revenue")
    giq_df = giq_df.fillna({"free_app_download":0, "paid_app_download":0})
    giq_df = giq_df.withColumn("download", giq_df.free_app_download + giq_df.paid_app_download)

    #compare
    s1=giq_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])
    s2=est_unified_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])

    diff = s1.union(s2).subtract(s1.intersect(s2))
    if diff.count()>0:
        print "{}: FAIL".format(date)
        print diff.show(2)
    else:
        print  "{}: PASS".format(date)
    
    
def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


date_list = get_date_list("2010-07-04/", "2020-02-08/")
for date in date_list:
    try:
        compare(date)
    except Exception, e:
        print "{}: ERROR".format(date) 



In [0]:
%md

1. fill na > 0逻辑, download有, 但是 revenue没有. 所以会出现 download =0 , revenue=null的情况
2. 正在跑


In [0]:


sql = "device_code='ios-phone' and country_code='NA' and genre_id=207"
s1.filter(sql).show()
s2.filter(sql).show()


In [0]:


import pandas as pd

test_df1 = pd.DataFrame({'id': [1,2,3], 'download':[1,2,3]})
test_df2 = pd.DataFrame({'id': [1,1,2,3,4], 'id2':[11,111,22,33,44], 'nickname':['aa','aa','bb','cc','dd']})


result_df = test_df1.merge(test_df2, on='id', how='outer')

print result_df


In [0]:

from pyspark.sql import Row
import pyspark.sql.functions as F 


df = spark.createDataFrame( [Row(1, [1,2,3])], ['id', 'genre_id'])
df.show()
df = df.withColumn('genre_id', F.explode('genre_id'))
df.show()


In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2019-12-27/

#                           PRE app-tech.store.app-est.v1/



In [0]:
%%sh
